In [1]:
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import ssl
ssl._create_default_https_context = ssl._create_unverified_context # Used to bypass SSL certificate verification
os.makedirs('preview', exist_ok=True)# Make directory for augmentation of images
NUM_TO_AUGMENT=5

#CIFAR_10 is a set  of 60K images 32x32 pixels on Flatten 3 channels
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32
#constant
BATCH_SIZE = 128
NB_EPOCH = 20
NB_CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = RMSprop()
#load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
#augementing
print("Augmenting training set images...")
# convert to categorical
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)
# float and normalization
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
# network
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()
# train
model.compile(loss='categorical_crossentropy', optimizer=OPTIM, metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=NB_EPOCH, validation_split=VALIDATION_SPLIT, verbose=VERBOSE)
score = model.evaluate(X_test, Y_test, batch_size=BATCH_SIZE, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])
#save model
model_json = model.to_json()
open('cifar10_architecture.json', 'w').write(model_json)
#And the weights learned by our deep network on the training set
model.save_weights('cifar10_weights.h5', overwrite=True)
datagen = ImageDataGenerator( rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
xtas, ytas = [], []
for i in range(X_train.shape[0]):
    num_aug = 0
    x = X_train[i] # (3, 32, 32)
    x = x.reshape((1,) + x.shape) # (1, 3, 32, 32)
for x_aug in datagen.flow(x, batch_size=1, save_to_dir='preview', save_prefix='cifar', save_format='jpeg'):
    if num_aug >= NUM_TO_AUGMENT:
        break
    xtas.append(x_aug[0])
    num_aug += 1
#fit the datagen
datagen.fit(X_train)
#train
history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE), samples_per_epoch=X_train.shape[0], epochs=NB_EPOCH, verbose=VERBOSE)
score = model.evaluate(X_test, Y_test, batch_size=BATCH_SIZE, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Augmenting training set images...


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)       

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:88: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=20, verbose=1, steps_per_epoch=390)`


Epoch 1/20
390/390 [==============================] - 189s 485ms/step - loss: 1.4448 - accuracy: 0.5036
Epoch 2/20
390/390 [==============================] - 186s 476ms/step - loss: 1.3452 - accuracy: 0.5333
Epoch 3/20
390/390 [==============================] - 192s 493ms/step - loss: 1.3112 - accuracy: 0.5466
Epoch 4/20
390/390 [==============================] - 187s 479ms/step - loss: 1.2832 - accuracy: 0.5568
Epoch 5/20
390/390 [==============================] - 154s 395ms/step - loss: 1.2697 - accuracy: 0.5603
Epoch 6/20
390/390 [==============================] - 113s 290ms/step - loss: 1.2603 - accuracy: 0.5653
Epoch 7/20
390/390 [==============================] - 117s 300ms/step - loss: 1.2398 - accuracy: 0.5730
Epoch 8/20
390/390 [==============================] - 112s 288ms/step - loss: 1.2350 - accuracy: 0.5746
Epoch 9/20
390/390 [==============================] - 114s 291ms/step - loss: 1.2283 - accuracy: 0.5798
Epoch 10/20
390/390 [==============================] - 115s 295m

### Ethical and Privacy Implications

The algorithm I just created can distinguish more realistic images between animals or vehicles. While this doesn’t pose a serious threat or ethical dilemma, the algorithm can be trained on different types of images. I’d consider my face to be  “Personally Identifiable Information (PII), which [can] uniquely [identify] a person” (Dorschel, 2019). The algorithm's ability to distinguish between more realistic images raises concerns about its potential use in facial recognition technology. If trained on a dataset containing images of people's faces, the algorithm could potentially be used for facial recognition purposes. This raises significant ethical and privacy implications. 

In fact, there is a company, called Clearview AI, that uses facial recognition software. “You take a picture of a person, upload it and get to see public photos of that person, along with links to where those photos appeared” (Hill, 2020). Clearview’s system has a database of more than 3 billion images that were “scraped from Facebook, YouTube, Venmo and millions of other websites” (Hill, 2020). Scraping images from certain websites, like Facebook, is a violation of some websites’ terms of service, which is unethical and may also be illegal. This is why transparency is crucial in Machine Learning algorithms. Transparent algorithms ensure that developers and users are aware of how data is collected, used, and processed. This transparency helps ensure that AI systems adhere to ethical guidelines and respect the rights and privacy of individuals. The widespread deployment of facial recognition algorithms could result in increased surveillance and control by governments, corporations, or other entities. This could threaten civil liberties, freedom of expression, and democratic principles. 

“Absent a very strong privacy law, we’re all screwed” (Hill, 2020).

#### References:


Dorschel, A. (2019, April 24). Rethinking Data Privacy: Impact of Machine Learning. Medium. https://medium.com/luminovo/data-privacy-in-machine-learning-a-technical-deep-dive-f7f0365b1d60 



Hill, K. (2020, January 21). The Secretive Company That Might End Privacy as We Know It. International New York Times.